# 競馬予想アプリ - スクレイピング & 予測

このノートブックはGoogle Colabで実行し、以下を行います：
1. レースデータのスクレイピング
2. 予測の実行
3. Supabaseへのデータ保存

## 使用方法
1. 左上の「ファイル」→「ドライブにコピー」でコピーを作成
2. シークレットに `SUPABASE_URL` と `SUPABASE_SERVICE_KEY` を設定
3. セルを順番に実行

## 1. 環境セットアップ

In [ ]:
# 必要なパッケージをインストール
!pip install -q supabase requests beautifulsoup4 lxml pandas numpy lightgbm scikit-learn

In [ ]:
# Supabase認証情報の設定
# 方法1: Google Colabシークレットを使用（推奨）
try:
    from google.colab import userdata
    SUPABASE_URL = userdata.get('SUPABASE_URL')
    SUPABASE_KEY = userdata.get('SUPABASE_SERVICE_KEY')
    print("Colabシークレットから認証情報を取得しました")
except:
    # 方法2: 直接入力（テスト用）
    SUPABASE_URL = "https://your-project.supabase.co"  # @param {type:"string"}
    SUPABASE_KEY = "your-service-role-key"  # @param {type:"string"}
    print("手動入力の認証情報を使用します")

# 接続テスト
from supabase import create_client
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
print(f"Supabase接続: {SUPABASE_URL[:30]}...")

## 2. スクレイピング関数の定義

In [ ]:
import requests
import time
import re
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup
from typing import Optional, List, Dict

# スクレイピング設定
SCRAPE_INTERVAL = 1.5  # リクエスト間隔（秒）
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "ja,en-US;q=0.9,en;q=0.8",
}

# JRA競馬場コード
JRA_COURSE_CODES = {"01", "02", "03", "04", "05", "06", "07", "08", "09", "10"}
COURSE_NAMES = {
    "01": "札幌", "02": "函館", "03": "福島", "04": "新潟",
    "05": "東京", "06": "中山", "07": "中京", "08": "京都",
    "09": "阪神", "10": "小倉",
}

session = requests.Session()
session.headers.update(HEADERS)
last_request_time = 0

def fetch_html(url: str) -> str:
    """HTMLを取得（レート制限付き）"""
    global last_request_time
    elapsed = time.time() - last_request_time
    if elapsed < SCRAPE_INTERVAL:
        time.sleep(SCRAPE_INTERVAL - elapsed)
    
    response = session.get(url, timeout=30)
    last_request_time = time.time()
    
    if "EUC-JP" in response.text[:500] or "euc-jp" in response.text[:500].lower():
        response.encoding = "euc-jp"
    else:
        response.encoding = response.apparent_encoding or "utf-8"
    
    return response.text

def is_jra_race(race_id: str) -> bool:
    """JRAレースかどうか判定"""
    if len(race_id) >= 6:
        return race_id[4:6] in JRA_COURSE_CODES
    return False

print("スクレイピング関数を定義しました")

In [ ]:
def scrape_race_list(target_date: date, jra_only: bool = True) -> List[Dict]:
    """指定日のレース一覧を取得"""
    date_str = target_date.strftime("%Y%m%d")
    url = f"https://db.netkeiba.com/race/list/{date_str}/"
    
    html = fetch_html(url)
    soup = BeautifulSoup(html, "lxml")
    
    races = []
    seen_ids = set()
    
    for link in soup.find_all("a", href=True):
        href = link.get("href", "")
        match = re.search(r"/race/(\d{12})/", href)
        if match:
            race_id = match.group(1)
            if race_id not in seen_ids:
                if jra_only and not is_jra_race(race_id):
                    continue
                seen_ids.add(race_id)
                races.append({
                    "race_id": race_id,
                    "date": target_date.isoformat(),
                    "race_name": link.get_text(strip=True),
                })
    
    return races

def scrape_race_detail(race_id: str) -> Dict:
    """レース詳細を取得"""
    url = f"https://db.netkeiba.com/race/{race_id}/"
    html = fetch_html(url)
    soup = BeautifulSoup(html, "lxml")
    
    # レース情報
    race_data = {
        "race_id": race_id,
        "course": COURSE_NAMES.get(race_id[4:6], ""),
        "race_number": int(race_id[10:12]) if len(race_id) >= 12 else 0,
    }
    
    # レース名
    title_elem = soup.select_one(".racedata h1, .data_intro h1")
    if title_elem:
        race_data["race_name"] = title_elem.get_text(strip=True)
    
    # 距離・馬場
    race_data_elem = soup.select_one(".racedata, .data_intro")
    if race_data_elem:
        text = race_data_elem.get_text()
        
        distance_match = re.search(r"(\d+)m", text)
        if distance_match:
            race_data["distance"] = int(distance_match.group(1))
        
        if "芝" in text:
            race_data["track_type"] = "芝"
        elif "ダート" in text or "ダ" in text:
            race_data["track_type"] = "ダート"
        
        condition_match = re.search(r"(芝|ダ)\s*:\s*(良|稍重|重|不良)", text)
        if condition_match:
            race_data["condition"] = condition_match.group(2)
        
        for grade in ["(G1)", "(G2)", "(G3)", "(L)", "オープン", "3勝", "2勝", "1勝", "新馬", "未勝利"]:
            if grade in text:
                race_data["grade"] = grade.replace("(", "").replace(")", "")
                break
    
    # 出走馬
    entries = []
    table = soup.select_one("table.race_table_01")
    if table:
        for row in table.select("tr")[1:]:
            entry = parse_entry_row(row)
            if entry:
                entries.append(entry)
    
    race_data["entries"] = entries
    return race_data

def parse_entry_row(row) -> Optional[Dict]:
    """出走馬の行をパース"""
    cells = row.select("td")
    if len(cells) < 10:
        return None
    
    entry = {}
    
    # 着順
    try:
        result_text = cells[0].get_text(strip=True)
        if result_text.isdigit():
            entry["result"] = int(result_text)
    except:
        pass
    
    # 枠番
    try:
        entry["frame_number"] = int(cells[1].get_text(strip=True))
    except:
        pass
    
    # 馬番
    try:
        entry["horse_number"] = int(cells[2].get_text(strip=True))
    except:
        return None
    
    # 馬名・ID
    horse_link = cells[3].select_one("a")
    if horse_link:
        href = horse_link.get("href", "")
        match = re.search(r"/horse/(\d+)", href)
        if match:
            entry["horse_id"] = match.group(1)
        entry["horse_name"] = horse_link.get_text(strip=True)
    
    # 性齢
    try:
        sex_age = cells[4].get_text(strip=True)
        if sex_age:
            entry["sex"] = sex_age[0]
    except:
        pass
    
    # 斤量
    try:
        entry["weight"] = float(cells[5].get_text(strip=True))
    except:
        pass
    
    # 騎手
    jockey_link = cells[6].select_one("a")
    if jockey_link:
        href = jockey_link.get("href", "")
        match = re.search(r"/jockey/(?:result/recent/)?(\d+)", href)
        if match:
            entry["jockey_id"] = match.group(1)
        entry["jockey_name"] = jockey_link.get_text(strip=True)
    
    # タイム
    try:
        time_text = cells[7].get_text(strip=True)
        if time_text:
            entry["finish_time"] = time_text
    except:
        pass
    
    # オッズ
    try:
        if len(cells) > 12:
            odds_text = cells[12].get_text(strip=True)
            if odds_text:
                entry["odds"] = float(odds_text)
    except:
        pass
    
    # 人気
    try:
        if len(cells) > 13:
            pop_text = cells[13].get_text(strip=True)
            if pop_text.isdigit():
                entry["popularity"] = int(pop_text)
    except:
        pass
    
    return entry if entry.get("horse_number") else None

print("レーススクレイピング関数を定義しました")

## 3. Supabase保存関数

In [ ]:
def save_horse(horse_data: Dict) -> bool:
    """馬をSupabaseに保存"""
    try:
        # 既存チェック
        existing = supabase.table("horses").select("horse_id").eq("horse_id", horse_data["horse_id"]).execute()
        
        if existing.data:
            supabase.table("horses").update(horse_data).eq("horse_id", horse_data["horse_id"]).execute()
        else:
            supabase.table("horses").insert(horse_data).execute()
        return True
    except Exception as e:
        print(f"馬の保存エラー: {e}")
        return False

def save_jockey(jockey_data: Dict) -> bool:
    """騎手をSupabaseに保存"""
    try:
        existing = supabase.table("jockeys").select("jockey_id").eq("jockey_id", jockey_data["jockey_id"]).execute()
        
        if existing.data:
            supabase.table("jockeys").update(jockey_data).eq("jockey_id", jockey_data["jockey_id"]).execute()
        else:
            supabase.table("jockeys").insert(jockey_data).execute()
        return True
    except Exception as e:
        print(f"騎手の保存エラー: {e}")
        return False

def save_race(race_data: Dict) -> bool:
    """レースをSupabaseに保存"""
    try:
        entries = race_data.pop("entries", [])
        
        # レース保存
        existing = supabase.table("races").select("race_id").eq("race_id", race_data["race_id"]).execute()
        
        if existing.data:
            supabase.table("races").update(race_data).eq("race_id", race_data["race_id"]).execute()
        else:
            supabase.table("races").insert(race_data).execute()
        
        # 出走馬保存
        for entry in entries:
            # 馬を保存
            if "horse_id" in entry:
                horse_data = {
                    "horse_id": entry["horse_id"],
                    "name": entry.get("horse_name", "不明"),
                    "sex": entry.get("sex", "不"),
                    "birth_year": 2020,  # 仮
                }
                save_horse(horse_data)
            
            # 騎手を保存
            if "jockey_id" in entry:
                jockey_data = {
                    "jockey_id": entry["jockey_id"],
                    "name": entry.get("jockey_name", "不明"),
                }
                save_jockey(jockey_data)
            
            # 出走馬保存
            entry_data = {
                "race_id": race_data["race_id"],
                "horse_id": entry.get("horse_id"),
                "jockey_id": entry.get("jockey_id"),
                "frame_number": entry.get("frame_number"),
                "horse_number": entry["horse_number"],
                "weight": entry.get("weight"),
                "odds": entry.get("odds"),
                "popularity": entry.get("popularity"),
                "result": entry.get("result"),
                "finish_time": entry.get("finish_time"),
            }
            
            # 既存チェック
            existing = supabase.table("entries").select("id").eq("race_id", race_data["race_id"]).eq("horse_number", entry["horse_number"]).execute()
            
            if existing.data:
                supabase.table("entries").update(entry_data).eq("id", existing.data[0]["id"]).execute()
            else:
                supabase.table("entries").insert(entry_data).execute()
        
        return True
    except Exception as e:
        print(f"レースの保存エラー: {e}")
        return False

print("Supabase保存関数を定義しました")

## 4. スクレイピング実行

In [ ]:
# 対象日を設定
target_date = date.today()  # 今日
# target_date = date(2024, 12, 22)  # 特定の日付

print(f"対象日: {target_date}")
print(f"JRAレースのみ取得します")

In [ ]:
# レース一覧を取得
races = scrape_race_list(target_date, jra_only=True)
print(f"\n{len(races)}件のレースが見つかりました")

for race in races[:5]:  # 最初の5件を表示
    print(f"  - {race['race_id']}: {race['race_name']}")

In [ ]:
# 全レースの詳細を取得して保存
from tqdm.notebook import tqdm

success_count = 0
error_count = 0

for race_info in tqdm(races, desc="レース取得中"):
    try:
        race_detail = scrape_race_detail(race_info["race_id"])
        race_detail["date"] = race_info["date"]
        
        if save_race(race_detail):
            success_count += 1
        else:
            error_count += 1
            
    except Exception as e:
        print(f"\nエラー: {race_info['race_id']} - {e}")
        error_count += 1

print(f"\n完了: 成功 {success_count}件, エラー {error_count}件")

## 5. 予測の実行（オプション）

予測を実行するには、事前に学習済みモデルをGoogle Driveにアップロードしておく必要があります。

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# モデルパスを設定
MODEL_PATH = "/content/drive/MyDrive/keiba_models/model_v1.pkl"  # 適宜変更

import os
if os.path.exists(MODEL_PATH):
    print(f"モデルが見つかりました: {MODEL_PATH}")
else:
    print(f"モデルが見つかりません: {MODEL_PATH}")
    print("Google Driveにモデルをアップロードしてください")

In [ ]:
# 予測関数（簡易版）
import pickle
import pandas as pd
import numpy as np

def load_model(model_path: str):
    """モデルをロード"""
    with open(model_path, 'rb') as f:
        return pickle.load(f)

def save_prediction(race_id: str, results: Dict) -> bool:
    """予測結果をSupabaseに保存"""
    try:
        prediction_data = {
            "race_id": race_id,
            "model_version": "colab_v1",
            "results_json": results,
        }
        supabase.table("predictions").insert(prediction_data).execute()
        return True
    except Exception as e:
        print(f"予測保存エラー: {e}")
        return False

print("予測関数を定義しました")

## 6. データ確認

In [ ]:
# Supabaseのデータ件数を確認
tables = ["races", "entries", "horses", "jockeys", "predictions"]

print("Supabaseデータ件数:")
for table in tables:
    try:
        result = supabase.table(table).select("*", count="exact").execute()
        print(f"  {table}: {result.count}件")
    except Exception as e:
        print(f"  {table}: エラー - {e}")

In [ ]:
# 今日のレースを確認
today = date.today().isoformat()
result = supabase.table("races").select("*").eq("date", today).execute()

print(f"\n{today}のレース: {len(result.data)}件")
for race in result.data:
    print(f"  - {race['race_id']}: {race['course']} {race['race_number']}R {race.get('race_name', '')}")

---

## 使い方のまとめ

1. **毎日のスクレイピング**: セル1-4を実行
2. **予測の実行**: セル5を実行（要モデル）
3. **データ確認**: セル6を実行

### スケジュール実行

毎日自動でスクレイピングを実行したい場合は、このノートブックをスケジュール実行に設定できます。

1. ノートブックを保存
2. 「編集」→「ノートブックの設定」
3. 「接続」→「Cloud Colab」でスケジュール設定